### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import Data

#### Tables we can ignore
seasons, races, lapTimes

In [2]:
results = pd.read_csv("../data/results.csv")
status = pd.read_csv("../data/status.csv")

sprints = pd.read_csv("../data/sprint_results.csv")

qualifying = pd.read_csv("../data/qualifying.csv")
qualifying = qualifying[['raceId', 'driverId', 'constructorId', 'q1', 'q2', 'q3']]

pitStops = pd.read_csv("../data/pit_stops.csv")
totalPitStops = pitStops
pitStops = pitStops[['raceId', 'driverId', 'milliseconds']]
pitStops = pd.DataFrame(pitStops.groupby(['raceId', 'driverId'])['milliseconds'].mean()).reset_index()
pitStops.columns = ['raceId', 'driverId', 'meanPitStopTime']

drivers = pd.read_csv("../data/drivers.csv")
drivers['driverName'] = drivers['forename'] + " " + drivers['surname']
drivers = drivers[['driverId', 'driverName']]

contructors = pd.read_csv("../data/constructors.csv")
contructors = contructors[['constructorId', 'name']]
contructors.columns = ['constructorId', 'constructorName']

races = pd.read_csv("../data/races.csv")
races = races[['raceId', 'name', 'year']]
races.columns = ['raceId', 'raceName', 'year']

In [3]:
results = results.merge(races, on=['raceId']) \
    .merge(pitStops, on=['raceId', 'driverId']) \
    .merge(drivers, on=['driverId']) \
    .merge(contructors, on=['constructorId']) \
    .merge(races, on=['raceId']) \
    .merge(status, on=['statusId'])

In [4]:
sprints = sprints.merge(drivers, on=['driverId']) \
    .merge(contructors, on=['constructorId']) \
    .merge(races, on=['raceId']) \
    .merge(status, on='statusId')

In [5]:
qualifying = qualifying.merge(drivers, on=['driverId']) \
    .merge(contructors, on=['constructorId']) \
    .merge(races, on=['raceId'])

In [6]:
# Aggregating Results
results = results[['resultId', 'raceId','driverId', 'constructorId',
                   'raceName_x', 'year_x','driverName', 'constructorName',
                   'grid','position','status','points', 'laps',
                   'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
                   'fastestLapSpeed','meanPitStopTime'
                   ]]
results.columns = ['resultId','raceId','driverId', 'constructorId',
                   'raceName', 'year','driverName', 'constructorName',
                   'grid','position','status','points', 'laps',
                   'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
                   'fastestLapSpeed','meanPitStopTime'
                   ]

In [7]:
# Aggregating Results
sprints = sprints[['resultId', 'raceId', 'driverId', 'constructorId',
                   'driverName', 'constructorName', 'raceName', 'year', 
                   'status','grid', 'position','points', 'laps', 'time','milliseconds',
                   'fastestLap', 'fastestLapTime', 'statusId']] 

qualifying = qualifying[['raceId', 'driverId', 'constructorId','raceName',
                   'driverName', 'constructorName', 'year','q1', 'q2', 'q3']]

### Adding current driver information

In [8]:
currentDrivers = ['Pierre Gasly','Lance Stroll','Lando Norris',
                  'Max Verstappen','Esteban Ocon','George Russell',
                  'Yuki Tsunoda','Charles Leclerc','Lewis Hamilton',
                  'Valtteri Bottas','Sergio Pérez','Kevin Magnussen',
                  'Nyck de Vries','Fernando Alonso','Guanyu Zhou',
                  'Logan Sargeant','Alexander Albon','Carlos Sainz',
                  'Oscar Piastri','Nico Hülkenberg', 'Logan Sargent']

currentConstructors = ['Mercedes', 'McLaren', 'Red Bull', 'Williams', 
                       'Ferrari','AlphaTauri', 'Haas F1 Team', 'Alpine F1 Team',
                       'Aston Martin', 'Alfa Romeo']

currentRaces = ['Bahrain Grand Prix', 'Saudi Arabian Grand Prix', 'Australian Grand Prix',
                'Azerbaijan Grand Prix', 'Miami Grand Prix', 'Emilia Romagna Grand Prix',
                'Monaco Grand Prix','Spanish Grand Prix', 'Canadian Grand Prix', 'Austrian Grand Prix',
                'British Grand Prix', 'Hungarian Grand Prix', 'Belgian Grand Prix',
                'Dutch Grand Prix', 'Italian Grand Prix', 'Singapore Grand Prix',
                'Japanese Grand Prix', 'Qatar Grand Prix', 'United States Grand Prix','Mexico City Grand Prix', 'São Paulo Grand Prix', 'Las Vegas Grand Prix', 'Abu Dhabi Grand Prix']

### Qualifying Points

In [10]:
qualifying.head(5)

,raceId,driverId,constructorId,raceName,driverName,constructorName,year,q1,q2,q3
0,18,1,1,Australian Grand Prix,Lewis Hamilton,McLaren,2008,1:26.572,1:25.187,1:26.714
1,18,5,1,Australian Grand Prix,Heikki Kovalainen,McLaren,2008,1:25.664,1:25.452,1:27.079
2,18,9,2,Australian Grand Prix,Robert Kubica,BMW Sauber,2008,1:26.103,1:25.315,1:26.869
3,18,2,2,Australian Grand Prix,Nick Heidfeld,BMW Sauber,2008,1:25.960,1:25.518,1:27.236
4,18,4,4,Australian Grand Prix,Fernando Alonso,Renault,2008,1:26.907,1:26.188,\N


In [11]:
def identify_qualifying_category(df):
    """Identifies the qualifying category for a group of drivers from the same constructor."""
    
    # Determine how many drivers are in the group
    num_drivers = df['driverName'].nunique()
    
    # Determine how many drivers reach Q2 and Q3
    num_q2 = (df['q2'] != '\\N').sum()
    num_q3 = (df['q3'] != '\\N').sum()
    
    # Identify the qualifying category based on the number of drivers that reach Q2 and Q3
    if num_q2 == 0:
        return -1
    elif num_q2 == 1:
        return 1
    elif num_q2 == 2 and num_q3 == 0:
        return 3
    elif num_q2 == 2 and num_q3 == 1:
        return 5
    elif num_q2 == 2 and num_q3 == 2:
        return 10

In [12]:
consQualRoundPoints = pd.DataFrame(
    qualifying.groupby(['constructorId', 'raceId']).apply(identify_qualifying_category)).reset_index()

consQualRoundPoints.columns = ['constructorId', 'raceId', 'qualifyingRoundsPoints']

In [13]:
# Calculate points
results['startingGridPoints'] = results['grid'].apply(lambda x: 11-x if (x <= 10 and x>0 ) else 0)


# Sum points by race and constructor
consGridPoints = results.groupby(['raceId', 'constructorId'])['startingGridPoints'].sum().reset_index()

In [14]:
consQualRoundPoints = consQualRoundPoints.merge(consGridPoints, on=['raceId', 'constructorId'])

In [15]:
consQualRoundPoints.head(5)

,constructorId,raceId,qualifyingRoundsPoints,startingGridPoints
0,1,841,10,16
1,1,842,10,16
2,1,843,10,17
3,1,844,10,12
4,1,845,10,14


### Sprints Points

In [16]:
sprints.head(5)

,resultId,raceId,driverId,constructorId,driverName,constructorName,raceName,year,status,grid,position,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,Max Verstappen,Red Bull,British Grand Prix,2021,Finished,2,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,Lewis Hamilton,Mercedes,British Grand Prix,2021,Finished,1,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,Valtteri Bottas,Mercedes,British Grand Prix,2021,Finished,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,13,1061,8,51,Kimi Räikkönen,Alfa Romeo,British Grand Prix,2021,Finished,17,13,0,17,+50.677,1589103,12,1:32.139,1
4,15,1061,841,51,Antonio Giovinazzi,Alfa Romeo,British Grand Prix,2021,Finished,14,15,0,17,+53.225,1591651,14,1:32.457,1


In [17]:
sprints.position = sprints.position.replace("\\N", 20).astype(int)
sprints['pos_diff'] = sprints.grid - sprints.position
sprints['sprintOvertakePoints'] = 0
sprints['sprintPoints'] = 0
sprints.loc[sprints['pos_diff'] > 0, 'sprintOvertakePoints'] += sprints['pos_diff']
sprints.loc[sprints['pos_diff'] < 0, 'sprintOvertakePoints'] -= abs(sprints['pos_diff'])
sprints.loc[sprints['grid']==0, 'sprintOvertakePoints'] = 0


sprints['sprintPoints'] = sprints['position'].apply(lambda x: 9-x if (x <= 8 and x>0 ) else 0)
sprints.loc[sprints.status!='Finished', 'sprintPoints'] = -20

In [18]:
sprints.fastestLapTime.replace("\\N", "2:50.000", inplace=True)
sprints['fastestLapTime'] = sprints['fastestLapTime'].str.replace(":", "").str.replace(".", "").astype(int)
fastest_lap_per_race = sprints.groupby('raceId')['fastestLapTime'].idxmin()
driver_with_fastest_lap_per_race = sprints.loc[fastest_lap_per_race, ['raceId', 'driverId', 'fastestLapTime']]
driver_with_fastest_lap_per_race['fastestSprintLapPoints'] = 5
sprints = sprints.merge(driver_with_fastest_lap_per_race, on=['raceId', 'driverId'], how='left')
sprints.fastestSprintLapPoints.fillna(0, inplace=True)

<ipython-input-18-574687b57679>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sprints['fastestLapTime'] = sprints['fastestLapTime'].str.replace(":", "").str.replace(".", "").astype(int)


In [19]:
consSprintPoints = sprints.groupby(['raceId', 'constructorId'])[
    ['sprintPoints', 'sprintOvertakePoints', 'fastestSprintLapPoints']].sum().reset_index()

In [20]:
sprints = sprints[['resultId', 'raceId', 'driverId', 'constructorId', 'driverName',
       'constructorName', 'raceName', 'year', 'status', 'grid', 'position', 'sprintOvertakePoints',
       'sprintPoints','fastestSprintLapPoints']]

In [21]:
consSprintPoints.head(5)

,raceId,constructorId,sprintPoints,sprintOvertakePoints,fastestSprintLapPoints
0,1061,1,7,2.0,0.0
1,1061,3,0,0.0,0.0
2,1061,6,5,-2.0,0.0
3,1061,9,-12,-14.0,0.0
4,1061,51,0,3.0,0.0


### Race points

In [22]:
results['racePoints'] = results['points'].astype(int)
results.position = results.position.replace("\\N", 20).astype(int)
results['pos_diff'] = results.grid - results.position
results['overtakePoints'] = 0
results.loc[results['pos_diff'] > 0, 'overtakePoints'] += results['pos_diff']
results.loc[results['pos_diff'] < 0, 'overtakePoints'] -= abs(results['pos_diff'])

In [23]:
results.fastestLapTime.replace("\\N", "2:50.000", inplace=True)
results['fastestLapTime'] = results['fastestLapTime'].str.replace(":", "").str.replace(".", "").astype(int)
fastest_lap_per_race = results.groupby('raceId')['fastestLapTime'].idxmin()
driver_with_fastest_lap_per_race = results.loc[fastest_lap_per_race, ['raceId', 'driverId', 'fastestLapTime']]
driver_with_fastest_lap_per_race['fastestLapPoints'] = 10
results = results.merge(driver_with_fastest_lap_per_race, on=['raceId', 'driverId'], how='left')
results.fastestLapPoints.fillna(0, inplace=True)

<ipython-input-23-d318561d0867>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  results['fastestLapTime'] = results['fastestLapTime'].str.replace(":", "").str.replace(".", "").astype(int)


In [24]:
# Calculate the points for each pitstop
totalPitStops['points'] = totalPitStops.groupby(['raceId'])['milliseconds'].rank(method='min', ascending=True).apply(lambda x: 10 if x == 1 else (5 if x == 2 else (3 if x == 3 else 0)))

# Calculate the total points for each driver per race
driver_points = totalPitStops.groupby(['raceId', 'driverId'])['points'].sum().reset_index()

# Sort the data by raceId and points
driver_points = driver_points.sort_values(by=['raceId', 'points'], ascending=[True, False])

driver_points.columns = ['raceId', 'driverId', 'pitStopPoints']
driver_points = driver_points.reset_index(drop=True)

In [25]:
conPitStopPoints = driver_points

In [26]:
conRacePoints = results.groupby(['raceId', 'constructorId'])[['racePoints']].sum().reset_index()

### Combining Summary Data - Constructors

In [27]:
constructorData = results[['raceId', 'constructorId', 'driverId', 'raceName', 'constructorName', 'driverName']].reset_index(drop=True)

In [28]:
constructorData = constructorData \
    .merge(conRacePoints, on=['raceId', 'constructorId'], how='left') \
    .merge(conPitStopPoints, on=['raceId', 'driverId'], how='left') \
    .merge(consSprintPoints, on=['raceId', 'constructorId'], how='left') \
    .merge(consQualRoundPoints, on=['raceId', 'constructorId'], how='left')

In [29]:
constructorData.sprintPoints.fillna(0, inplace=True)
constructorData.sprintOvertakePoints.fillna(0, inplace=True)
constructorData.fastestSprintLapPoints.fillna(0, inplace=True)

In [30]:
pitPointsAdded = pd.DataFrame(constructorData.groupby(['raceId', 'constructorId'])['pitStopPoints'].sum()).reset_index()

In [31]:
constructorData = constructorData[['raceId', 'constructorId', 'raceName', 'constructorName',
                                   'racePoints', 'sprintPoints','sprintOvertakePoints', 
                                   'fastestSprintLapPoints','qualifyingRoundsPoints', 'startingGridPoints']].drop_duplicates()

In [32]:
constructorData = constructorData.merge(pitPointsAdded, on=['raceId', 'constructorId'], how='left')\
    .merge(races[['raceId', 'year']], on='raceId', how='left')

In [33]:
constructorData[constructorData.constructorName.isin(currentConstructors)]

,raceId,constructorId,raceName,constructorName,racePoints,sprintPoints,sprintOvertakePoints,fastestSprintLapPoints,qualifyingRoundsPoints,startingGridPoints,pitStopPoints,year
0,841,9,Australian Grand Prix,Red Bull,35,0.0,0.0,0.0,10.0,18.0,3,2011
1,841,6,Australian Grand Prix,Ferrari,18,0.0,0.0,0.0,10.0,9.0,0,2011
2,841,1,Australian Grand Prix,McLaren,26,0.0,0.0,0.0,10.0,16.0,10,2011
4,842,9,Malaysian Grand Prix,Red Bull,37,0.0,0.0,0.0,10.0,18.0,15,2011
5,842,6,Malaysian Grand Prix,Ferrari,18,0.0,0.0,0.0,10.0,10.0,0,2011
...,...,...,...,...,...,...,...,...,...,...,...,...
2526,1032,6,Styrian Grand Prix,Ferrari,0,0.0,0.0,0.0,5.0,0.0,0,2020
2527,1044,210,Turkish Grand Prix,Haas F1 Team,0,0.0,0.0,0.0,-1.0,0.0,0,2020
2528,1081,6,Azerbaijan Grand Prix,Ferrari,0,0.0,0.0,0.0,10.0,10.0,0,2022
2529,971,1,Bahrain Grand Prix,McLaren,0,0.0,0.0,0.0,-1.0,0.0,0,2017


### Combining Summary Data - Drivers

In [34]:
results = results[['raceId', 'driverId', 'constructorId', 'raceName', 'year',
       'driverName', 'constructorName','status', 'startingGridPoints',
       'racePoints', 'overtakePoints', 'fastestLapPoints']]

In [35]:
sprints = sprints[['raceId', 'driverId', 'constructorId', 'driverName',
                   'constructorName', 'raceName', 'year','sprintOvertakePoints', 
                   'sprintPoints', 'fastestSprintLapPoints']]

In [36]:
driverResults = results.merge(sprints, 
                              on=['raceId', 'driverId', 'constructorId', 
                                  'raceName', 'year', 'driverName', 'constructorName'], how='left')

In [37]:
driverResults = driverResults[driverResults.driverName.isin(currentDrivers)]

In [38]:
driverResults.fillna(0, inplace=True)

In [39]:
missingPoints = pd.DataFrame(driverResults.groupby(
    ['raceId', 'constructorId'])[['overtakePoints', 'fastestLapPoints']].sum()).reset_index()

In [40]:
constructorData = constructorData.merge(missingPoints, on=['raceId', 'constructorId'], how='left')

In [41]:
constructorData.fillna(0, inplace=True)

### Building the Linear Algebra Model

In [42]:
driverResults['totalPoints'] = driverResults[['startingGridPoints', 'racePoints','overtakePoints', 
     'fastestLapPoints', 'sprintOvertakePoints', 'sprintPoints', 
     'fastestSprintLapPoints']].sum(axis=1)

In [43]:
avgDriverPoints = pd.DataFrame(driverResults[driverResults.year.isin([2023])].groupby(
    ['driverName']).totalPoints.mean()).reset_index()

In [44]:
constructorData['totalPoints'] = constructorData[['racePoints',
       'sprintPoints', 'sprintOvertakePoints', 'fastestSprintLapPoints',
       'qualifyingRoundsPoints', 'startingGridPoints', 'pitStopPoints',
       'overtakePoints', 'fastestLapPoints']].sum(axis=1)

In [45]:
avgConstructorPoints = pd.DataFrame(constructorData[constructorData.year.isin([2023])] \
    .groupby(['constructorName']).totalPoints.mean()).reset_index()

In [46]:
driverResults[driverResults.year==2023].head(10)

,raceId,driverId,constructorId,raceName,year,driverName,constructorName,status,startingGridPoints,racePoints,overtakePoints,fastestLapPoints,sprintOvertakePoints,sprintPoints,fastestSprintLapPoints,totalPoints
1330,1098,815,9,Bahrain Grand Prix,2023,Sergio Pérez,Red Bull,Finished,9,18,0.0,0.0,0.0,0.0,0.0,27.0
1331,1098,830,9,Bahrain Grand Prix,2023,Max Verstappen,Red Bull,Finished,10,25,0.0,0.0,0.0,0.0,0.0,35.0
1332,1098,832,6,Bahrain Grand Prix,2023,Carlos Sainz,Ferrari,Finished,7,12,0.0,0.0,0.0,0.0,0.0,19.0
1333,1098,4,117,Bahrain Grand Prix,2023,Fernando Alonso,Aston Martin,Finished,6,15,2.0,0.0,0.0,0.0,0.0,23.0
1334,1098,840,117,Bahrain Grand Prix,2023,Lance Stroll,Aston Martin,Finished,3,8,2.0,0.0,0.0,0.0,0.0,13.0
1335,1098,1,131,Bahrain Grand Prix,2023,Lewis Hamilton,Mercedes,Finished,4,10,2.0,0.0,0.0,0.0,0.0,16.0
1336,1098,847,131,Bahrain Grand Prix,2023,George Russell,Mercedes,Finished,5,6,-1.0,0.0,0.0,0.0,0.0,10.0
1337,1098,842,214,Bahrain Grand Prix,2023,Pierre Gasly,Alpine F1 Team,Finished,0,2,11.0,0.0,0.0,0.0,0.0,13.0
1338,1098,848,3,Bahrain Grand Prix,2023,Alexander Albon,Williams,Finished,0,1,5.0,0.0,0.0,0.0,0.0,6.0
1339,1098,822,51,Bahrain Grand Prix,2023,Valtteri Bottas,Alfa Romeo,Finished,0,4,4.0,0.0,0.0,0.0,0.0,8.0


In [47]:
avgConstructorPoints

,constructorName,totalPoints
0,Alfa Romeo,10.333333
1,AlphaTauri,13.666667
2,Alpine F1 Team,12.333333
3,Aston Martin,40.666667
4,Ferrari,27.666667
5,Haas F1 Team,10.333333
6,McLaren,13.333333
7,Mercedes,36.333333
8,Red Bull,79.666667
9,Williams,10.666667


In [48]:
avgDriverPoints

,driverName,totalPoints
0,Alexander Albon,1.500000
1,Carlos Sainz,10.333333
2,Charles Leclerc,1.000000
3,Esteban Ocon,-1.666667
4,Fernando Alonso,23.000000
5,George Russell,6.666667
6,Guanyu Zhou,5.000000
7,Kevin Magnussen,1.666667
8,Lance Stroll,7.666667
9,Lando Norris,3.666667


### Building a Pulp model

In [62]:
# Create the constructor variables with their respective costs and points
constructorsCost = {
    "Alfa Romeo": 6.1,
    "AlphaTauri": 6.3,
    "Alpine F1 Team": 9.9,
    "Aston Martin": 6.9,
    "Ferrari": 22.1,
    "Haas F1 Team": 5.2,
    "McLaren": 9,
    "Mercedes": 25.1,
    "Red Bull": 27.5,
    "Williams": 4.9
}

# Create the driver variables with their respective costs
driverCosts = {
    "Alexander Albon": 5.3,
    "Carlos Sainz": 17.2,
    "Charles Leclerc": 21.1,
    "Esteban Ocon": 9.2,
    "Fernando Alonso": 8.9,
    "George Russell": 18.5,
    "Guanyu Zhou": 4.9,
    "Kevin Magnussen": 6.6,
    "Lance Stroll": 7.6,
    "Lando Norris": 11.2,
    "Lewis Hamilton": 23.8,
    "Max Verstappen": 27.2,
    "Nico Hülkenberg": 4.1,
    "Nyck de Vries": 4.9,
    "Oscar Piastri": 6.8,
    "Pierre Gasly": 8.1,
    "Sergio Pérez": 18.3,
    "Valtteri Bottas": 7.7,
    "Yuki Tsunoda": 4.8,
    "Logan Sargeant": 3.8
}

costCap = 101.7


currentDrivers = list(driverCosts.keys())
currentConstructors = list(constructorsCost.keys())

team = {
    "Alfa Romeo": ["Valtteri Bottas", "Guanyu Zhou"],
    "AlphaTauri": ["Nyck de Vries", "Yuki Tsunoda"],
    "Alpine F1 Team": ["Pierre Gasly", "Esteban Ocon"],
    "Aston Martin": ["Fernando Alonso", "Lance Stroll"],
    "Ferrari": ["Charles Leclerc", "Carlos Sainz"],
    "Haas F1 Team": ["Nico Hülkenberg", "Kevin Magnussen"],
    "McLaren": ["Lando Norris", "Oscar Piastri"],
    "Mercedes": ["Lewis Hamilton", "George Russell"],
    "Red Bull": ["Max Verstappen", "Sergio Pérez"],
    "Williams": ["Alexander Albon", "Logan Sargeant"],
}


In [63]:
scoresDB = []

for driver_name in currentDrivers:
    
    # Get average scores
    constructorPoints = dict(zip(avgConstructorPoints['constructorName'], avgConstructorPoints['totalPoints']))
    driverPoints = dict(zip(avgDriverPoints['driverName'], avgDriverPoints['totalPoints']))
    
    
    # Double the points of the selected driver in the driversPoints dictionary
    if driver_name in driverPoints.keys():
        old_points = driverPoints[driver_name]
        driverPoints[driver_name] = old_points * 2
        print(f"{driver_name}'s points have been doubled from {old_points} to {driverPoints[driver_name]}")
    else:
        print(f"{driver_name} is not a valid driver")

    # Add the old points to the corresponding constructor in the constructorsPoints dictionary
    for constructor, drivers in team.items():
        if driver_name in drivers:
            old_construct_score = constructorPoints[constructor]
            constructorPoints[constructor] += old_points
            print(f"{constructor}'s points have been doubled from {old_construct_score} to {constructorPoints[constructor]}")
            
    # Create the 'prob' variable to contain the problem data
    from pulp import *

    prob = LpProblem("Driver and Constructor Selection", LpMaximize)
    
    # Define the decision variables
    d = LpVariable.dicts("Driver", currentDrivers, cat=LpBinary)
    c = LpVariable.dicts("Constructor", currentConstructors, cat=LpBinary)
    
    # Define the objective function
    prob += pulp.lpSum(
        [d[i] * driverPoints[i] for i in currentDrivers]) + pulp.lpSum(
        [c[j] * constructorPoints[j] for j in currentConstructors])
    
    # Add constraints
    prob += pulp.lpSum([d[i] for i in currentDrivers]) == 5, "Select 5 drivers"
    prob += pulp.lpSum([c[j] for j in currentConstructors]) == 2, "Select 2 constructors"
    prob += pulp.lpSum(
        [d[i] * driverCosts[i] for i in currentDrivers]) + pulp.lpSum([c[j] * constructorsCost[j] for j in currentConstructors]) <= costCap, "Budget Constraint"
    
    # Solve the problem
    prob.solve()
    

    potentialScore = 0

    driverList = []
    constructorList = []

    # Print the solution
    print("Selected Drivers:")
    for i in currentDrivers:
        if d[i].varValue > 0:
            print(" - ", i)
            driverList.append(i)

    print("\nSelected Constructors:")
    for j in currentConstructors:
        if c[j].varValue > 0:
            print(" - ", j)
            constructorList.append(j)

    print("Total Points:", pulp.value(prob.objective))
    

    score = {
        "drivers": driverList, 
        "constructors": constructorList, 
        "potentialScore":  pulp.value(prob.objective), 
        "drs": driver_name
    }
    scoresDB.append(score)

Alexander Albon's points have been doubled from 1.5 to 3.0
Williams's points have been doubled from 10.666666666666666 to 12.166666666666666
Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Max Verstappen
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 219.33333333333331
Carlos Sainz's points have been doubled from 10.333333333333334 to 20.666666666666668
Ferrari's points have been doubled from 27.666666666666668 to 38.0
Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Max Verstappen
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 219.33333333333331
Charles Leclerc's points have been doubled from 1.0 to 2.0
Ferrari's points have been doubled from 27.666666666666668 to 28.666666666666668
Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Max Verstappen
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 219

/Users/ritwikdiv/opt/miniconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Max Verstappen
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 234.66666666666669
Lando Norris's points have been doubled from 3.6666666666666665 to 7.333333333333333
McLaren's points have been doubled from 13.333333333333334 to 17.0
Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Max Verstappen
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 219.33333333333331
Lewis Hamilton's points have been doubled from 19.666666666666668 to 39.333333333333336
Mercedes's points have been doubled from 36.333333333333336 to 56.0
Selected Drivers:
 -  Fernando Alonso
 -  Guanyu Zhou
 -  Lance Stroll
 -  Lewis Hamilton
 -  Sergio Pérez

Selected Constructors:
 -  Aston Martin
 -  Red Bull
Total Points: 221.33333333333331
Max Verstappen's points have been doubled from 37.333333333333336 to 74.66666666666667
Red Bull's points have 

# Best Team

In [76]:
solution = pd.DataFrame(scoresDB).sort_values(by="potentialScore",ascending=False).iloc[0]

In [77]:
print("\nDrivers to pick: ")
print(solution.drivers)

print("\nConstructors to pick: ")
print(solution.constructors)

print("\nPotential Score: ")
print(solution.potentialScore)

print("\nDRS Pick :")
print(solution.drs)


Drivers to pick: 
['Fernando Alonso', 'Guanyu Zhou', 'Lance Stroll', 'Max Verstappen', 'Sergio Pérez']

Constructors to pick: 
['Aston Martin', 'Red Bull']

Potential Score: 
294.0

DRS Pick :
Max Verstappen


In [78]:
pd.DataFrame(scoresDB).sort_values(by="potentialScore",ascending=False)

,drivers,constructors,potentialScore,drs
11,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, M...","[Aston Martin, Red Bull]",294.000000,Max Verstappen
16,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, M...","[Aston Martin, Red Bull]",271.333333,Sergio Pérez
4,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, M...","[Aston Martin, Red Bull]",265.333333,Fernando Alonso
8,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, M...","[Aston Martin, Red Bull]",234.666667,Lance Stroll
6,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, M...","[Aston Martin, Red Bull]",224.333333,Guanyu Zhou
18,"[Fernando Alonso, Lance Stroll, Max Verstappen...","[Aston Martin, Red Bull]",221.666667,Yuki Tsunoda
10,"[Fernando Alonso, Guanyu Zhou, Lance Stroll, L...","[Aston Martin, Red Bull]",221.333333,Lewis Hamilton
19,"[Fernando Alonso, Lance Stroll, Max Verstappen...","[Aston Martin, Red Bull]",221.000000,Logan Sargeant
15,"[Fernando Alonso, Max Verstappen, Pierre Gasly...","[Aston Martin, Red Bull]",220.333333,Pierre Gasly
13,"[Fernando Alonso, Lance Stroll, Max Verstappen...","[Aston Martin, Red Bull]",220.333333,Nyck de Vries
